In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain
from langchain_openai.chat_models import ChatOpenAI

In [ ]:
load_dotenv(override=True)

os.environ.get("OPENAI_API_KEY")

In [ ]:
file = "D:\Visual Code\RAG\pdf\entrecomp.pdf"
loader = PyPDFLoader(file)
document = loader.load()

In [ ]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [ ]:
chain = load_qa_chain(llm=llm, chain_type="stuff", verbose=True)